In [1]:
import os
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.metrics import mean_squared_error, mean_absolute_error
from utilities import cal_score, cal_mape

In [2]:
stack_idx = '17'
models = '1-31'
use_test_kfold = set([2, 7, 8, 12, 13])

is_per_area = True
add_intercept = True

### Read CV predictions and test

In [3]:
def parse_models(exp):
    exp_split = exp.split(',')
    idx_models = []
    for e in exp_split:
        if '-' in e:
            n0, n1 = e.split('-')
            idx_models.extend(list(range(int(n0), int(n1)+1, 1)))
        else:
            idx_models.append(int(e))
    return idx_models

In [4]:
idx_models = parse_models(models)
print(idx_models)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31]


In [5]:
files_in_output = [f for f in os.listdir('output/') if os.path.isfile('output/'+f)]
files_cv = {idx: [f for f in files_in_output 
                  if f.startswith('model-%02d-' % idx) and f.endswith('cv.csv')][0] 
            for idx in idx_models}
files_test_one = {idx: [f for f in files_in_output 
                        if f.startswith('model-%02d-' % idx) and f.endswith('test-one.csv')][0]
                  for idx in idx_models}
files_test_kf = {idx: [f for f in files_in_output 
                       if f.startswith('model-%02d-' % idx) and f.endswith('test-kfold.csv')][0]
                 for idx in idx_models}

In [6]:
for k in files_cv: 
    print('%2d'%k, files_cv[k])
    print('%2d'%k, files_test_kf[k])
    print('%2d'%k, files_test_one[k])

 1 model-01-lgb-cv.csv
 1 model-01-lgb-test-kfold.csv
 1 model-01-lgb-test-one.csv
 2 model-02-keras-search-cv.csv
 2 model-02-keras-search-test-kfold.csv
 2 model-02-keras-search-test-one.csv
 3 model-03-lgb-feats-selection-cv.csv
 3 model-03-lgb-feats-selection-test-kfold.csv
 3 model-03-lgb-feats-selection-test-one.csv
 4 model-04-lgb-PCA-cv.csv
 4 model-04-lgb-PCA-test-kfold.csv
 4 model-04-lgb-PCA-test-one.csv
 5 model-05-lgb-wo-per-area-cv.csv
 5 model-05-lgb-wo-per-area-test-kfold.csv
 5 model-05-lgb-wo-per-area-test-one.csv
 6 model-06-lgb-lr0.001-cv.csv
 6 model-06-lgb-lr0.001-test-kfold.csv
 6 model-06-lgb-lr0.001-test-one.csv
 7 model-07-keras-embedding-cv.csv
 7 model-07-keras-embedding-test-kfold.csv
 7 model-07-keras-embedding-test-one.csv
 8 model-08-keras-search-long-cv.csv
 8 model-08-keras-search-long-test-kfold.csv
 8 model-08-keras-search-long-test-one.csv
 9 model-09-lgb-feats-selection-75-cv.csv
 9 model-09-lgb-feats-selection-75-test-kfold.csv
 9 model-09-lgb-fea

#### Load area

In [7]:
df_train = pd.read_csv('dataset/train.csv')
df_test = pd.read_csv('dataset/test.csv')

In [8]:
cv = df_train[['building_id', 'building_area', 'total_price']]
test = df_test[['building_id', 'building_area']]

In [9]:
print('CV predictions:')
print(len(idx_models))
for i, idx_model in enumerate(idx_models):
    f = files_cv[idx_model]
    print(f)
#    print('No. {} file: {}'.format(i, f))
    df = pd.read_csv('output/'+f)
    
    cv = pd.merge(cv, df[['building_id', 'total_price_predict']], on='building_id')
    
    cv = cv.rename(columns = {'total_price_predict': 'pred_{}'.format(idx_model)})
    cv[f'log_pred_{idx_model}'] = np.log1p(cv[f'pred_{idx_model}'])
    cv[f'log_parea_pred_{idx_model}'] = np.log1p( cv[f'pred_{idx_model}'] / cv['building_area'] )

CV predictions:
31
model-01-lgb-cv.csv
model-02-keras-search-cv.csv
model-03-lgb-feats-selection-cv.csv
model-04-lgb-PCA-cv.csv
model-05-lgb-wo-per-area-cv.csv
model-06-lgb-lr0.001-cv.csv
model-07-keras-embedding-cv.csv
model-08-keras-search-long-cv.csv
model-09-lgb-feats-selection-75-cv.csv
model-10-lgb-feats-selection-75-lr-0.001-cv.csv
model-11-rf-cv.csv
model-12-predict-keras-search-prelu-cv.csv
model-13-predict-keras-he_uni-cv.csv
model-14-lgb-feats-selection-75-lr-0.001-rand-cv.csv
model-15-lgb-feats-selection-75-lr-0.001-rand323-cv.csv
model-16-lgb-feats-selection-68-lr-0.001-mix5-cv.csv
model-17-lgb-feats-selection-70-lr-0.001-mix5-cv.csv
model-18-lgb-feats-selection-70-lr-0.001-p5-cv.csv
model-19-lgb-search-bins-lr-0.0005-cv.csv
model-20-lgb-lr-0.0008-mix5-cv.csv
model-21-lgb-wo-per-area-long-cv.csv
model-22-lgb-wo-per-area-long-2-cv.csv
model-23-lgb-binary-cv.csv
model-24-lgb-binary-augment-cv.csv
model-25-lgb-search-bins-lr-0.0005-250-cv.csv
model-26-lgb-search-bins-lr-0.000

In [10]:
cv['log_total_price'] = np.log1p(cv['total_price'])
cv['log_parea_total_price'] = np.log1p( cv['total_price'] / cv['building_area'] )

print('Test predictions:')
for i, idx in enumerate(idx_models):
    f = files_test_kf[idx] if idx in use_test_kfold else files_test_one[idx]
    print('No. {} file: {}'.format(i, f))
    df = pd.read_csv('output/'+f)

    test = pd.merge(test, df[['building_id','total_price']], on='building_id')
        
    test = test.rename(columns = {'total_price': 'pred_{}'.format(idx)})
    test[f'log_pred_{idx}'] = np.log1p(test[f'pred_{idx}'])
    test[f'log_parea_pred_{idx}'] = np.log1p( test[f'pred_{idx}'] / test['building_area'] )

Test predictions:
No. 0 file: model-01-lgb-test-one.csv
No. 1 file: model-02-keras-search-test-kfold.csv
No. 2 file: model-03-lgb-feats-selection-test-one.csv
No. 3 file: model-04-lgb-PCA-test-one.csv
No. 4 file: model-05-lgb-wo-per-area-test-one.csv
No. 5 file: model-06-lgb-lr0.001-test-one.csv
No. 6 file: model-07-keras-embedding-test-kfold.csv
No. 7 file: model-08-keras-search-long-test-kfold.csv
No. 8 file: model-09-lgb-feats-selection-75-test-one.csv
No. 9 file: model-10-lgb-feats-selection-75-lr-0.001-test-one.csv
No. 10 file: model-11-rf-test-one.csv
No. 11 file: model-12-predict-keras-search-prelu-test-kfold.csv
No. 12 file: model-13-predict-keras-he_uni-test-kfold.csv
No. 13 file: model-14-lgb-feats-selection-75-lr-0.001-rand-test-one.csv
No. 14 file: model-15-lgb-feats-selection-75-lr-0.001-rand323-test-one.csv
No. 15 file: model-16-lgb-feats-selection-68-lr-0.001-mix5-test-one.csv
No. 16 file: model-17-lgb-feats-selection-70-lr-0.001-mix5-test-one.csv
No. 17 file: model-18-l

In [11]:
display(cv.head())
display(test.head())

,building_id,building_area,total_price,pred_1,log_pred_1,log_parea_pred_1,pred_2,log_pred_2,log_parea_pred_2,pred_3,...,log_pred_29,log_parea_pred_29,pred_30,log_pred_30,log_parea_pred_30,pred_31,log_pred_31,log_parea_pred_31,log_total_price,log_parea_total_price
0,e3mMIMR3JJqCaXz1,3.418175,6.476038e+05,6.331552e+05,13.358472,12.129369,717209.94,13.483125,12.254022,6.656891e+05,...,13.349368,12.120265,6.687262e+05,13.413132,12.184028,6.789011e+05,13.428232,12.199129,13.381036,12.151933
1,LgwzgklNvy4QCtq5,4.041309,3.321452e+06,3.064324e+06,14.935338,13.538770,2899842.20,14.880167,13.483600,3.079196e+06,...,14.961478,13.564910,3.180461e+06,14.972537,13.575969,2.996046e+06,14.912804,13.516237,15.015913,13.619345
2,ucIR2NLLsC3T650L,5.584279,9.570885e+06,9.827776e+06,16.100723,14.380769,9766813.00,16.094501,14.374546,9.814852e+06,...,16.094058,14.374103,9.739343e+06,16.091684,14.371729,9.840726e+06,16.102040,14.382085,16.074236,14.354282
3,jre1pJhcQj91Kdky,13.563031,1.421501e+07,1.255350e+07,16.345510,13.738164,12699800.00,16.357097,13.749750,1.255981e+07,...,16.351248,13.743901,1.242492e+07,16.335215,13.727868,1.246321e+07,16.338292,13.730945,16.469809,13.862462
4,rQpYpY9nRG7X5mmr,4.688108,7.627120e+05,1.215194e+06,14.010415,12.465389,2012610.50,14.514944,12.969916,1.128419e+06,...,13.936413,12.391388,1.227169e+06,14.020221,12.475195,1.159105e+06,13.963160,12.418134,13.544637,11.999613


,building_id,building_area,pred_1,log_pred_1,log_parea_pred_1,pred_2,log_pred_2,log_parea_pred_2,pred_3,log_pred_3,...,log_parea_pred_28,pred_29,log_pred_29,log_parea_pred_29,pred_30,log_pred_30,log_parea_pred_30,pred_31,log_pred_31,log_parea_pred_31
0,X5gsdTWGS3W7JJQB,3.418175,1.526912e+07,16.541343,15.312236,12470072.0,16.338842,15.109735,1.531685e+07,16.544464,...,15.284537,1.458851e+07,16.495745,15.266638,1.341823e+07,16.412125,15.183018,1.235658e+07,16.329699,15.100592
1,BTshNOJyKHnT2YIT,7.726227,3.924241e+06,15.182684,13.138065,3916552.2,15.180723,13.136104,3.977095e+06,15.196062,...,13.138282,3.930381e+06,15.184247,13.139628,3.931973e+06,15.184652,13.140033,3.933268e+06,15.184982,13.140363
2,dhdymr0lV8N5kZOT,12.170581,1.096127e+07,16.209879,13.710858,11912735.0,16.293119,13.794098,1.084967e+07,16.199646,...,13.683307,1.053625e+07,16.170332,13.671312,1.221239e+07,16.317962,13.818941,1.039900e+07,16.157220,13.658199
3,VEwyGGMcD56w5BOc,2.252256,6.155550e+06,15.632865,14.820933,5940670.0,15.597333,14.785400,6.015238e+06,15.609807,...,14.799625,5.922934e+06,15.594343,14.782410,5.957837e+06,15.600218,14.788286,5.944207e+06,15.597928,14.785996
4,wmUeMoJZfsqaSX9b,5.813985,1.062995e+06,13.876602,12.116340,1088488.1,13.900301,12.140039,1.027248e+06,13.842395,...,12.131217,1.092658e+06,13.904125,12.143863,1.110080e+06,13.919943,12.159682,1.140621e+06,13.947084,12.186822


### Check models scores

In [12]:
for i, idx_model in enumerate(idx_models):
    print('%2d'%i, 'model-%02d'%idx_model, '%.6f'%cal_score(cv['total_price'], cv[f'pred_{idx_model}']))

 0 model-01 5870.873059
 1 model-02 5400.852164
 2 model-03 5877.873452
 3 model-04 5713.867808
 4 model-05 5724.869598
 5 model-06 5886.873769
 6 model-07 5171.836449
 7 model-08 5514.858826
 8 model-09 5872.873118
 9 model-10 5897.873845
10 model-11 5075.838018
11 model-12 5486.856963
12 model-13 5506.858055
13 model-14 5908.873901
14 model-15 5900.873836
15 model-16 5907.874126
16 model-17 5905.874165
17 model-18 5908.874297
18 model-19 5911.874156
19 model-20 5908.874040
20 model-21 5758.870702
21 model-22 5752.870671
22 model-23 5852.871357
23 model-24 5866.873835
24 model-25 5908.874238
25 model-26 5918.873998
26 model-27 5892.873194
27 model-28 5901.874202
28 model-29 5884.873848
29 model-30 5864.872101
30 model-31 5783.868923


In [13]:
cv['constant_1'] = 1
test['constant_1'] = 1

if is_per_area:
    cols_opt = [f'log_parea_pred_{idx}' for idx in idx_models]
else:
    cols_opt = [f'log_pred_{idx}' for idx in idx_models]

if add_intercept:
    cols_opt.append('constant_1')

### Define opt function

In [14]:
def objective(x, idx, metric, best_score, best_coeffs, verbose):
    cv_pred_final = cv.loc[idx,cols_opt].dot(x)
    
    if is_per_area:
        cv_pred_final = np.expm1(cv_pred_final) * cv.loc[idx,'building_area']
    else:
        cv_pred_final = np.expm1(cv_pred_final)

    score = cal_score(cv.loc[idx,'total_price'], cv_pred_final)
    if score > best_score[metric]:
        best_score[metric] = score
        best_coeffs[metric] = x.copy()
        if verbose:
            print('find better score:')
            print('score: ', score)
            print('coeffs: ', x)
            print()
    
    if metric == 'mape':
        return cal_mape(cv.loc[idx,'total_price'], cv_pred_final)
    elif metric == 'mse':
        return mean_squared_error(cv.loc[idx,'total_price'], cv_pred_final)
    elif metric == 'mae':
        return mean_absolute_error(cv.loc[idx,'total_price'], cv_pred_final)
    elif metric == 'smooth':
        return cal_score_smooth(cv.loc[idx,'total_price'], cv_pred_final)
    else:
        raise Exception('metric unknown: {}'.format(metric))
#    return 1 - (cal_score(cv['total_price'], cv_pred_final)/10000)

### CV

In [15]:
from sklearn.model_selection import KFold

cv = cv.reset_index(drop=True)
#cv = cv.head(100)

len_x = len(cols_opt)
rev_len_x = 1/len_x
x0s = [[1/len_x for i in range(len_x)],
       [0, 0, 0.05263157894736842, 0, 0.05263157894736842, 0, 0, 0.05263157894736842, 0, 0, 0, 0.05263157894736842,
        0.05263157894736842, 0, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842,
        0, 0.05263157894736842, 0, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842,
        0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0, 0.05263157894736842,
        0],
       [0 for i in range(len_x)],
       [0.1 for i in range(len_x)],
       [0.2 for i in range(len_x)],
       [0.5 for i in range(len_x)],
       [np.random.randn()+1/len_x for i in range(len_x)],
       [np.random.randn()+1/len_x for i in range(len_x)],
       [np.random.randn()+1/len_x for i in range(len_x)] ]

score_list = []

kf = KFold(shuffle= True)
for idx_train, idx_val in kf.split(cv):

    best_score = {}
    best_coeffs = {}

    for metric in ['mape']:
    #for metric in ['mape', 'mae', 'mse']:
        best_score[metric] = 0
        best_coeffs[metric] = []
        for x0 in x0s:
#            print('Optimizing with init x0: {}'.format(x0))
#            print()
            minimize(objective, x0, args=(idx_train, metric, best_score, best_coeffs, False), tol=1e-4)
    
    val_pred_final = cv.loc[idx_val, cols_opt].dot(best_coeffs['mape'])
    if is_per_area:
        val_pred_final = np.expm1(val_pred_final) * cv.loc[idx_val, 'building_area']
    else:
        val_pred_final = np.expm1(val_pred_final)
    score = cal_score(cv.loc[idx_val, 'total_price'], val_pred_final)
    
    score_list.append(score)

print('CV score ?: {}; {}'.format(np.mean(score_list), score_list))

/usr/local/lib/python3.7/site-packages/sklearn/model_selection/_split.py:426: FutureWarning: You should specify a value for 'n_splits' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(NSPLIT_WARNING, FutureWarning)
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in expm1
  """
/usr/local/lib/python3.7/site-packages/scipy/optimize/optimize.py:670: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in expm1
  """
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in expm1
  """
/usr/local/lib/python3.7/site-packages/scipy/optimize/optimize.py:670: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/local

CV score ?: 5956.210229434066; [5932.875207888342, 5964.878090273449, 5970.877390140405]


### Optimize

In [16]:
best_score = {}
best_coeffs = {}

len_x = len(cols_opt)
rev_len_x = 1/len_x
x0s = [[1/len_x for i in range(len_x)],
       [0, 0, 0.05263157894736842, 0, 0.05263157894736842, 0, 0, 0.05263157894736842, 0, 0, 0, 0.05263157894736842,
        0.05263157894736842, 0, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842,
        0, 0.05263157894736842, 0, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842,
        0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0.05263157894736842, 0, 0.05263157894736842, 0],
       [0 for i in range(len_x)],
       [0.1 for i in range(len_x)],
       [0.2 for i in range(len_x)],
       [0.5 for i in range(len_x)],
       [np.random.randn()+1/len_x for i in range(len_x)],
       [np.random.randn()+1/len_x for i in range(len_x)],
       [np.random.randn()+1/len_x for i in range(len_x)] ]

for metric in ['mape']:
#for metric in ['mape', 'mae', 'mse']:
    best_score[metric] = 0
    best_coeffs[metric] = []
    for x0 in x0s:
        print('Optimizing with init x0: {}'.format(x0))
        print()
        minimize(objective, x0, args=(cv.index, metric, best_score, best_coeffs, True), tol=1e-4)

Optimizing with init x0: [0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125, 0.03125]

find better score:
score:  354.6741209262715
coeffs:  [0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125 0.03125
 0.03125 0.03125 0.03125 0.03125 0.03125]

find better score:
score:  354.67412105163646
coeffs:  [0.03125001 0.03125    0.03125    0.03125    0.03125    0.03125
 0.03125    0.03125    0.03125    0.03125    0.03125    0.03125
 0.03125    0.03125    0.03125    0.03125    0.03125    0.03125
 0.03125    0.03125    0.03125    0.03125    0.03125    0.03125
 0.03125    0.03125    0.03125    0.03125    0.03125    0.

find better score:
score:  5954.876634291534
coeffs:  [ 0.02810136  0.03612786  0.03265302  0.0298861   0.04679009  0.030354
  0.03349348  0.04466527  0.02900619  0.03065032 -0.07404608  0.03587842
  0.0344696   0.03147456  0.03054581  0.03259192  0.03305291  0.03489356
  0.03326434  0.03211842  0.04593814  0.04637895  0.03910617  0.04859408
  0.03517602  0.03159965  0.03557354  0.03402744  0.0349711   0.04326203
  0.04192901 -0.04727026]

find better score:
score:  5954.876634291538
coeffs:  [ 0.02810136  0.03612786  0.03265302  0.02988608  0.04679009  0.030354
  0.03349348  0.04466529  0.02900619  0.03065032 -0.07404608  0.03587842
  0.0344696   0.03147456  0.03054581  0.03259192  0.03305291  0.03489356
  0.03326434  0.03211842  0.04593814  0.04637895  0.03910617  0.04859408
  0.03517602  0.03159965  0.03557354  0.03402744  0.0349711   0.04326203
  0.04192901 -0.04727026]

find better score:
score:  5954.876634291542
coeffs:  [ 0.02810136  0.03612786  0.03265302  0.02988608  0.046790

find better score:
score:  5967.877166045031
coeffs:  [-0.15625784  0.01778226  0.14449381  0.05341404  0.09295079  0.0019135
  0.02524757  0.09418727 -0.08209633 -0.09760331 -0.10245647  0.01324579
  0.02622267  0.09972387 -0.13890907  0.042903    0.14016543  0.25505775
  0.02409596 -0.34334855 -0.01843085  0.08280336 -0.17375977  0.21606895
  0.05726476  0.01143893  0.3286502  -0.06694676  0.0968628   0.29917526
  0.06297503 -0.10392216]

find better score:
score:  5967.877166259798
coeffs:  [-0.15470403  0.01676224  0.14482236  0.05321699  0.09153166  0.00225057
  0.02577132  0.09482049 -0.07970661 -0.09727083 -0.102025    0.01358615
  0.02607182  0.0990794  -0.13836944  0.0414198   0.1378656   0.2511216
  0.02361488 -0.34112544 -0.01837635  0.08211892 -0.17167182  0.21830667
  0.05682871  0.01083252  0.3322831  -0.06669301  0.09787831  0.29360866
  0.06304621 -0.10472648]

find better score:
score:  5967.877166409793
coeffs:  [-0.15475356  0.01573304  0.14480683  0.05377385  0.0917

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in expm1
  """
/usr/local/lib/python3.7/site-packages/scipy/optimize/optimize.py:670: RuntimeWarning: invalid value encountered in double_scalars
  grad[k] = (f(*((xk + d,) + args)) - f0) / d[k]
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: overflow encountered in expm1
  """


Optimizing with init x0: [-0.6100338113350916, -0.2030458414128532, 0.27365025265424203, 0.48790835909289215, -1.1892823479734076, -0.13325877664137703, -0.2017701901319427, 1.2781823480027414, -0.6336413988834858, -1.461682496181627, 1.9698389631485427, 0.5734119169593367, -1.608280641130845, -1.6981306340962743, 1.7410384115545297, 0.2788707237836989, 0.6214269714024983, 0.06859300519397646, 1.2089318056047726, 0.382089815384848, 0.889493593034382, -1.8771655390251467, 0.8688167396283558, 1.1988507203655918, -0.9738489199893658, -0.2417013411181837, 1.501309417977364, 1.5971740723736572, 0.3330964847184318, 0.7790449475517037, -1.0968514346171434, 1.261943552584088]

Optimizing with init x0: [-0.5467550038568518, -0.4605198050273249, -0.08507176770400192, -0.2553311027593755, -1.1042073380350663, 0.9371018083899806, -0.33746103077940387, 0.1190814775403279, 0.07883820719979433, 0.5383576990129886, -0.9751166720426614, 0.8496151533967968, 0.17271637830962397, -0.08463314054404064, -0.

In [17]:
display(best_score)
display(best_coeffs)

{'mape': 5969.877031790558}

{'mape': array([-0.07633914,  0.01908046,  0.04715564,  0.09626171,  0.07163536,
        -0.03531958,  0.02398886,  0.1087664 , -0.05732052, -0.04163452,
        -0.09156344, -0.00331682,  0.02616816, -0.02545347,  0.00539145,
         0.03669926,  0.04110954,  0.07321346, -0.01387079, -0.00637282,
         0.01974991,  0.07909496,  0.02987207,  0.20586191,  0.04363977,
         0.03359134,  0.17086198,  0.01896291,  0.04929472,  0.08801076,
         0.06971908, -0.10491713])}

### Compute submission

In [ ]:
test_pred_final = pd.DataFrame({'building_id': test['building_id']})

test_pred_final['total_price'] = test.loc[:, cols_opt].dot(best_coeffs['mape'])

if is_per_area:
    test_pred_final['total_price'] = np.expm1(test_pred_final['total_price']) * test['building_area'] 
else:
    test_pred_final['total_price'] = np.expm1(test_pred_final['total_price'])
    
test_pred_final['total_price'] = np.clip(test_pred_final['total_price'], 0, None)

if is_per_area:
    test_pred_final.to_csv('output/stack_spopt-parea_{}_{}.csv'.format(stack_idx, models), index=False)
else:
    test_pred_final.to_csv('output/stack_spopt_{}_{}.csv'.format(stack_idx, models), index=False)

### Plots

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(cv['log_parea_total_price'], bins=100, label='train true', normed=True)
plt.hist(np.log1p(test_pred_final['total_price'] / test['building_area']), bins=100, label='test',
         normed=True, alpha=0.7)
plt.xlabel('log(price/area + 1)'); plt.ylabel('ratio')
plt.legend(); plt.grid(); plt.show()

In [ ]:
plt.hist(cv['log_total_price'], bins=100, label='train true', normed=True)
plt.hist(np.log1p(test_pred_final['total_price']), bins=100, label='test', normed=True, alpha=0.7)
plt.xlabel('log(price + 1)'); plt.ylabel('ratio')
plt.legend(); plt.grid(); plt.show()

In [ ]:
plt.hist(np.log1p(cv['building_area']), bins=100, label='train', normed=True)
plt.hist(np.log1p(test['building_area']), bins=100, label='test', normed=True, alpha=0.7)
plt.xlabel('log(building_area + 1)'); plt.ylabel('ratio'); plt.yscale('log')
plt.legend(); plt.grid(); plt.show()